<a href="https://colab.research.google.com/github/pomellonn/Risk_analytics/blob/main/hw_4_ft_school_npv_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Домашнее задание №4. Работа с NPV моделью

*Дата выдачи: 28.01.2026*

*Дедлайн: 02.02.2026 23:59*

### Задание №1: проанализировать чувствительномть модели к параметрам

Формат -- меняем параметры вверх-вниз, смотрим на NPV и объясняем, почему это происходит.

### Задание №2: добавить страховку в NPV модель
- Страховка стоит 0.5% от лимита
- Подключена у 40% клиентов в статусе CUR и у 60% в статусе DLQ
- Возможно наступление страхового случая с вероятностью 0.1%, при подключенной страховке мы должны возместить всю сумму текущей задолженности

Вам нужно внести изменения в код ниже. Где – подсказывать не будем, это часть задания. Добавьте комментарий с указанием места, котороые вы поменяли.

### Класс модели

In [1]:
import numpy as np

class NPVModel:

    avg_missed_payments = 1.5
    recovery = 0.50
    dlq_penalty_amount = 500
    oper_costs = 100
    collection_costs = 600
    tax_rate = 0.20
    discounting_rate = 0.30
    eq_req = 0.125
    cost_of_funds = 0.16
    dlq_ratio = np.ones(101)*0.20
    dlq_ratio[0] = 0
    acquisition_cost = 1000

    def __init__(self):
        pass

    def model_balance_calculations(self, amount, rate, term):
        '''
        Расчет модельных баланса, выплаченных процентов, регулярного платежа
        :param amount: Сумма кредита
        :param rate: Ставка
        :param term: Срок
        :return:
        balance : np.array(101) : остаток тела долга по кредиту на каждый месяц
        interest : np.array(101) : выплата по процентам каждый месяц
        regular_payment : float : размер регулярного платежа
        '''

        regular_payment = round(amount * (rate / 12) *\
                                (1 + (rate / 12)) / (1 - ((1 + rate / 12) ** (-term))))
        # График баланса и процентов
        balance = np.zeros(101)
        interest = np.zeros(101)
        balance[0] = amount

        for i in range(1, term + 1):
            int_payment = balance[i - 1] * rate / 12
            debt_payment = regular_payment - int_payment
            balance[i] = max(0, round(balance[i - 1] - debt_payment))
            interest[i] = int_payment
        return balance, interest, regular_payment

    def distribution_calc(self, portfolio_distribution, pd=None, pa=None):
        '''
        Расчет распределения по статусам
        :param portfolio_distribution: график распределения по статусам
        :return:
        cur_dist : np.array(101) : доля клиентов в статусе CUR на каждый месяц
        dlq_dist : np.array(101) : доля клиентов в статусе DLQ на каждый месяц
        act_dist : np.array(101) : доля клиентов в статусе ACT на каждый месяц
        def_dist : np.array(101) : доля клиентов в статусе DEF на каждый месяц
        clo_dist : np.array(101) : доля клиентов в статусе CLO на каждый месяц
        '''

        cur_dist = portfolio_distribution['CUR']
        dlq_dist = portfolio_distribution['DLQ']
        act_dist = cur_dist + dlq_dist
        def_dist = portfolio_distribution['DEF']
        clo_dist = portfolio_distribution['CLO']


        return cur_dist, dlq_dist, act_dist, def_dist, clo_dist

    def cur_balance_calc(self, model_balance, cur_dist):
        '''
        Расчет модельного баланса в статусе CUR
        :param model_balance: плановый график баланса
        :param cur_dist: доля клиентов в статусе CUR на каждый месяц
        :return:
        principal_balance_cur : np.array(101) : principal balance в статусе CUR
        gross_balance_cur : np.array(101) : gross balance в статусе CUR
        '''
        principal_balance_cur = model_balance*cur_dist
        gross_balance_cur = model_balance*cur_dist
        return principal_balance_cur, gross_balance_cur

    def dlq_balance_calc(self, model_balance, regular_payment, dlq_dist):
        '''
        Расчет модельного баланса в статусе DLQ
        :param model_balance: плановый график баланса
        :param regular_payment: размер регулярного платежа
        :param dlq_dist: доля клиентов в статусе DLQ на каждый месяц
        :return:
        principal_balance_dlq : np.array(101) : principal balance в статусе DLQ
        gross_balance_dlq : np.array(101) : gross balance в статусе DLQ
        '''
        principal_balance_dlq = np.append(model_balance[1:], model_balance[-1])*dlq_dist
        gross_balance_dlq = (model_balance + regular_payment*self.avg_missed_payments)*dlq_dist
        return principal_balance_dlq, gross_balance_dlq

    def act_balance_calc(self, principal_balance_cur, principal_balance_dlq, gross_balance_cur, gross_balance_dlq):
        '''
        Расчет модельного баланса в статусе ACT
        :param principal_balance_cur: principal balance в статусе CUR
        :param principal_balance_dlq: principal balance в статусе DLQ
        :param gross_balance_cur: principal balance в статусе CUR
        :param gross_balance_dlq: principal balance в статусе DLQ
        :param dlq_dist: доля клиентов в статусе DLQ на каждый месяц
        :return:
        principal_balance_act : np.array(101) : principal balance в статусе ACT
        gross_balance_act : np.array(101) : gross balance в статусе ACT
        '''
        principal_balance_act = principal_balance_cur + principal_balance_dlq
        gross_balance_act = gross_balance_cur + gross_balance_dlq
        return principal_balance_act, gross_balance_act

    def def_balance_calc(self, model_balance, regular_payment, def_dist):
        '''
        Расчет модельного баланса в статусе DEF
        :param model_balance: плановый график баланса
        :param regular_payment: размер регулярного платежа
        :param def_dist: доля клиентов в статусе DEF на каждый месяц
        :return:
        principal_balance_def : np.array(101) : principal balance в статусе DEF
        gross_balance_def : np.array(101) : gross balance в статусе DEF
        '''
        principal_balance_def = np.zeros(101)
        gross_balance_def = np.zeros(101)

        def_dist_change = def_dist[4:] - def_dist[3:-1]
        principal_balance_def[4:] = np.cumsum(model_balance[:-4] * def_dist_change)
        gross_balance_def[4:] = np.cumsum((model_balance[4:] + 4 * regular_payment) * def_dist_change)

        return principal_balance_def, gross_balance_def

    def profit_calc(self, principal_balance_act, principal_balance_def, term, rate, dlq_dist):
        '''

        :param principal_balance_act: principal balance в статусе ACT
        :param principal_balance_def: principal balance в статусе DEF
        :param term: срок кредита
        :param rate: тавка по кредиту
        :param dlq_dist: доля клиентов в статусе DLQ на каждый месяц
        :return:
        profit : np.array(101) : доход на каждый месяц
        '''
        interest_profit = principal_balance_act*rate/12
        penatly_profit = dlq_dist*self.dlq_penalty_amount

        new_def_balance = np.append(principal_balance_def[1:], 0) - principal_balance_def
        recovery_profit = new_def_balance*self.recovery

        profit = interest_profit + recovery_profit + penatly_profit
        profit[term+1:] = 0

        return profit

    def loss_calc(self, gross_balance_act, principal_balance_def, term, act_dist, dlq_dist, def_dist):
        '''

        :param gross_balance_act: gross balance в статусе ACT
        :param principal_balance_def: principal balance в статусе DEF
        :param term: срок кредита
        :param act_dist: доля клиентов в статусе ACT на каждый месяц
        :param dlq_dist: доля клиентов в статусе DLQ на каждый месяц
        :param def_dist: доля клиентов в статусе DEF на каждый месяц
        :return:
        loss : np.array(101) : лосс на каждый месяц
        '''
        oper_loss = np.append(act_dist[1:], 0)*self.oper_costs
        loan_loss =  principal_balance_def - np.append(0, principal_balance_def[:-1])
        collection_loss = (np.append(dlq_dist[1:], 0) + np.append(def_dist[1:], 0))*self.collection_costs

        prev_gross_balance_act = np.append(gross_balance_act[0], gross_balance_act[1:])
        cost_of_funds_loss = prev_gross_balance_act*(1 - self.eq_req)*self.cost_of_funds/12

        loss = loan_loss + cost_of_funds_loss + oper_loss + collection_loss
        loss[term+1:] = 0

        return loss, loan_loss, cost_of_funds_loss, oper_loss, collection_loss

    def assets_liabilities_calc(self, gross_balance_act):
        '''

        :param gross_balance_act: gross balance в статусе ACT
        :return:
        assets : np.array(101) : активы в проекте
        eq_req_curve : np.array(101) : активы, обеспеченные капиталом
        fund_req_curve : np.array(101) : активы, обеспеченные фондами
        '''

        # assets
        assets = gross_balance_act

        #liabilities
        eq_req_curve = assets*self.eq_req
        fund_req_curve = assets*(1 - self.eq_req)

        return assets, eq_req_curve, fund_req_curve

    def niat_calc(self, profit, loss):
        '''
        Расчет NIAT
        :param profit: суммарный доход на каждый месяц
        :param loss: суммарный лосс на каждый месяц
        :return:
        nibt : np.array(101) : прибыль до налогообложения
        niat : np.array(101) : прибыль после налогообложения
        tax : np.array(101) : налог в каждом месяце
        '''

        nibt = profit - loss
        tax = nibt*self.tax_rate
        niat = nibt - tax

        return nibt, niat, tax

    def cashflow_calc(self, principal_balance_act, principal_balance_def, amount, profit,
                      cost_of_funds_loss, oper_costs, collection_costs, tax, niat, eq_req_curve, fund_req_curve):
        '''
        Расчет денежных потоков
        :param principal_balance_act: principal balance в статусе DEF
        :param principal_balance_def: principal balance в статусе DEF
        :param amount: срок кредита
        :param profit: доход на каждый месяц
        :param cost_of_funds_loss: лоссы на фондирование
        :param oper_costs: операционные расходы
        :param collection_costs: расходы на коллекшн
        :param tax: налог в каждом месяце
        :param niat: доход после налогообложения
        :param eq_req_curve: активы, обеспеченные капиталом
        :param fund_req_curve: активы, обеспеченные фондами
        :return:
        cf_to_client : np.array(101) : денежный поток к клиенту
        cf_to_shareholders : np.array(101) : денежный поток к акционерам
        cf_to_debtholders : np.array(101) : денежный поток к фондам
        cf_to_cost_and_tax : np.array(101) : денежный поток на косты и налоги
        '''
        # client
        delta_principal_balance_act = (np.append(0, principal_balance_act[:100]) - principal_balance_act)
        delta_principal_balance_def = (np.append(0, principal_balance_def[:100]) - principal_balance_def)
        repayments = delta_principal_balance_act - delta_principal_balance_def + profit
        cf_to_client = np.append(amount, -repayments[1:])

        # debtoholder
        fund_req_ch = fund_req_curve - np.append(fund_req_curve[0], fund_req_curve[:-1])
        fund_req_change = np.append(fund_req_curve[0], fund_req_ch[1:])
        cf_to_debtholders = cost_of_funds_loss - fund_req_change

        # shareholders
        eq_req_ch = eq_req_curve - np.append(eq_req_curve[0], eq_req_curve[:-1])
        cf_to_shareholders = niat - np.append(eq_req_curve[0], eq_req_ch[1:])

        # cost and tax
        cf_to_cost_and_tax = oper_costs + collection_costs + tax

        return cf_to_client, cf_to_shareholders, cf_to_debtholders, cf_to_cost_and_tax

    def npv_calc(self, amount, rate, term, portfolio_distribution, pd=None, pa=None):
        '''
        Расчет NPV
        :param amount: сумма кредита
        :param rate: ставка по кредиту
        :param term: срок кредита
        :param portfolio_distribution: график распределения по статусам
        :param pd: вероятность дефолта к 12-ому месяцу
        :param pa: вероятность полного досрочного погашения к 6-ому месяцу
        :return: npv: чистая приведенная стоимость кредита
        '''

        # Считаем балансы
        model_balance, interest, regular_payment =  self.model_balance_calculations(amount, rate, term)

        # Считаем распределение
        cur_dist, dlq_dist, act_dist, def_dist, clo_dist = self.distribution_calc(portfolio_distribution)

        # Считаем балансы
        principal_balance_cur, gross_balance_cur = self.cur_balance_calc(model_balance, cur_dist)
        principal_balance_dlq, gross_balance_dlq = self.dlq_balance_calc(model_balance, regular_payment, dlq_dist)
        principal_balance_act, gross_balance_act = self.act_balance_calc(principal_balance_cur, principal_balance_dlq, gross_balance_cur, gross_balance_dlq)
        principal_balance_def, gross_balance_def = self.def_balance_calc(model_balance, regular_payment, def_dist)

        # Считаем профиты
        profit = self.profit_calc(principal_balance_act, principal_balance_def, term, rate, dlq_dist)
        loss, loan_loss, cost_of_funds_loss, oper_loss, collection_loss = self.loss_calc(gross_balance_act, principal_balance_def,
                                                                                                                  term, act_dist, dlq_dist, def_dist)
        nibt, niat, tax = self.niat_calc(profit, loss)

        # Считаем активы и денежные потоки
        assets, eq_req_curve, fund_req_curve = self.assets_liabilities_calc(gross_balance_act)
        cf_to_client, cf_to_shareholders, cf_to_debtholders, cf_to_cost_and_tax =\
            self.cashflow_calc(principal_balance_act, principal_balance_def, amount, profit, cost_of_funds_loss, oper_loss, collection_loss,
                               tax, niat, eq_req_curve, fund_req_curve)


        # Считаем NPV
        disc_curve = np.array([1 / ((1 + self.discounting_rate) ** (i / 12.)) for i in range(101)])
        pv = np.round(np.sum(disc_curve * cf_to_shareholders)).astype(int)
        npv = pv - self.acquisition_cost

        return npv

### Пример использования модели

In [2]:
cur_dist = np.zeros(101)
dlq_dist = np.zeros(101)
clo_dist = np.zeros(101)
def_dist = np.zeros(101)

dlq_dist[1:12] += 0.05

def_dist[6:] += 0.03
def_dist[9:] += 0.03

def_dist[12:] += dlq_dist[11]

clo_dist[3:] += 0.10
clo_dist[6:] += 0.10
clo_dist[12:] = 1 - def_dist[12]

cur_dist = 1 - def_dist - clo_dist - dlq_dist

portfolio_distribution = {
    'CUR' : cur_dist,
    'DLQ' : dlq_dist,
    'DEF' : def_dist,
    'CLO' : clo_dist
         }

In [3]:
portfolio_distribution

{'CUR': array([1.  , 0.95, 0.95, 0.85, 0.85, 0.85, 0.72, 0.72, 0.72, 0.69, 0.69,
        0.69, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  ]),
 'DLQ': array([0.  , 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05,
        0.05, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
  

In [4]:
import plotly.express as px
import pandas as pd

df = pd.DataFrame({
    'CUR': cur_dist,
    'DLQ': dlq_dist,
    'DEF': def_dist,
    'CLO': clo_dist
})

df = df.reset_index().rename(columns={'index': 'Номер выписки'})
df = df.melt(id_vars='Номер выписки', value_vars=['DEF', 'DLQ', 'CUR', 'CLO'],
             var_name='Status', value_name='Доля')

fig = px.area(df, x='Номер выписки', y='Доля', color='Status',
              title='Распределение статусов по номерам выписки',
              color_discrete_map={
                  'DEF': '#D62728',
                  'DLQ': '#FF7F0E',
                  'CUR': '#2CA02C',
                  'CLO': '#1F77B4',
              },
              category_orders={'Status': ['DEF', 'DLQ', 'CUR', 'CLO']}
             )

fig.update_layout(height=600)
fig.update_xaxes(range=[0, 15])

fig.show()

In [5]:
model = NPVModel()
model.npv_calc(amount=500_000, term=12, rate=0.25, portfolio_distribution=portfolio_distribution)

np.int64(361)

## Задание 1


- Сумма кредита
- Срок кредита
- Ставка
- Доля дефолтников
- Доля полный закрытий
- Доля капитала в выдаче

### `Сумма кредита`



In [6]:
model.npv_calc(amount=700_000, term=12, rate=0.25, portfolio_distribution=portfolio_distribution)

np.int64(1267)

In [7]:
model.npv_calc(amount=300_000, term=12, rate=0.25, portfolio_distribution=portfolio_distribution)

np.int64(-545)

при увеличении суммы кредита NPV выше, при уменьшении - ниже. думаю это связано с тем, что проценты выплачиваемые заемщиком напрямую зависят от суммы кредита (выше сумма кредита - больше денег клиент выплачивает банку за кредит), а так как в нашем случае доля cur намного выше остальных, профит, зависящий от выплачиваемых процентов выше чем потери появляющиеся от дефолтников

### `Срок кредита`

In [8]:
for year in range(1, 7):
  print(year, model.npv_calc(amount=500_000, term=year*12, rate=0.25, portfolio_distribution=portfolio_distribution))

1 361
2 716
3 647
4 520
5 399
6 296


Выходит что при сроке кредита - 2 года, NPV становится выше, далее идет снижение. Мне кажется, это связано с снижением ценности денег (дисконтирование)

также распределение в portfolio_distribution рассчитано на год , поэтому тут годы рассчитывать не очень корректно

npv при изменении срока - от 3 до 12 месяцев

In [9]:
for months in range(3, 13):
  print(months, model.npv_calc(amount=500_000, term=months, rate=0.25, portfolio_distribution=portfolio_distribution))

3 2243
4 3276
5 7501
6 1746
7 2390
8 4961
9 1534
10 1923
11 4592
12 361


на 5 месяцев высоко, тк дефолтники появляются с 6 месяца (по текущему portfolio_distribution). а появление дефолтников - означает больший loss.



###`Ставка`

In [10]:
for r in range(10, 40):
  model = NPVModel()
  print(r, model.npv_calc(amount=500_000, term=12, rate=r*0.01, portfolio_distribution=portfolio_distribution))

10 -26944
11 -25140
12 -23333
13 -21524
14 -19713
15 -17899
16 -16082
17 -14264
18 -12443
19 -10620
20 -8794
21 -6967
22 -5138
23 -3307
24 -1474
25 361
26 2198
27 4036
28 5875
29 7717
30 9559
31 11403
32 13248
33 15094
34 16942
35 18790
36 20639
37 22489
38 24340
39 26191


Выше ставка - выше npv, тк профит зависит от процентов выплат. Но наблюдаются отрицательные значения, происходит при < 24%. То есть все возможные затраты банка больше получаемой прибыли, если ставка по кредиту < 24%

###`Доля дефолтников`





In [11]:
cur_dist_d = np.zeros(101)
dlq_dist_d = np.zeros(101)
clo_dist_d = np.zeros(101)
def_dist_d = np.zeros(101)

dlq_dist_d[1:12] += 0.05

def_dist_d[6:] += 0.03
def_dist_d[9:] += 0.03

def_dist_d[12:] += dlq_dist_d[11]
def_dist_d *= 0.5
clo_dist_d[3:] += 0.10
clo_dist_d[6:] += 0.10
clo_dist_d[12:] = 1 - def_dist_d[12]

cur_dist_d = 1 - def_dist_d - clo_dist_d - dlq_dist_d

portfolio_distribution_d_small = {
    'CUR' : cur_dist_d,
    'DLQ' : dlq_dist_d,
    'DEF' : def_dist_d,
    'CLO' : clo_dist_d
         }

In [12]:
portfolio_distribution_d_small['DEF']

array([0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.015, 0.015, 0.015,
       0.03 , 0.03 , 0.03 , 0.055, 0.055, 0.055, 0.055, 0.055, 0.055,
       0.055, 0.055, 0.055, 0.055, 0.055, 0.055, 0.055, 0.055, 0.055,
       0.055, 0.055, 0.055, 0.055, 0.055, 0.055, 0.055, 0.055, 0.055,
       0.055, 0.055, 0.055, 0.055, 0.055, 0.055, 0.055, 0.055, 0.055,
       0.055, 0.055, 0.055, 0.055, 0.055, 0.055, 0.055, 0.055, 0.055,
       0.055, 0.055, 0.055, 0.055, 0.055, 0.055, 0.055, 0.055, 0.055,
       0.055, 0.055, 0.055, 0.055, 0.055, 0.055, 0.055, 0.055, 0.055,
       0.055, 0.055, 0.055, 0.055, 0.055, 0.055, 0.055, 0.055, 0.055,
       0.055, 0.055, 0.055, 0.055, 0.055, 0.055, 0.055, 0.055, 0.055,
       0.055, 0.055, 0.055, 0.055, 0.055, 0.055, 0.055, 0.055, 0.055,
       0.055, 0.055])

In [13]:
model.npv_calc(amount=500_000, term=12, rate=0.25, portfolio_distribution=portfolio_distribution_d_small)

np.int64(5438)

In [14]:
cur_dist = np.zeros(101)
dlq_dist = np.zeros(101)
clo_dist = np.zeros(101)
def_dist = np.zeros(101)

dlq_dist[1:12] += 0.05

def_dist[6:] += 0.03
def_dist[9:] += 0.03

def_dist[12:] += dlq_dist[11]
def_dist *= 1.5
clo_dist[3:] += 0.10
clo_dist[6:] += 0.10
clo_dist[12:] = 1 - def_dist[12]

cur_dist = 1 - def_dist - clo_dist - dlq_dist

portfolio_distribution_big = {
    'CUR' : cur_dist,
    'DLQ' : dlq_dist,
    'DEF' : def_dist,
    'CLO' : clo_dist
         }

In [15]:
portfolio_distribution_big['DEF']

array([0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.045, 0.045, 0.045,
       0.09 , 0.09 , 0.09 , 0.165, 0.165, 0.165, 0.165, 0.165, 0.165,
       0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165,
       0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165,
       0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165,
       0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165,
       0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165,
       0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165,
       0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165,
       0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165,
       0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.165,
       0.165, 0.165])

In [16]:
model.npv_calc(amount=500_000, term=12, rate=0.25, portfolio_distribution=portfolio_distribution_big)

np.int64(-4715)

Доля дефолтников больше доли дефолтников из примера в 1.5 раза - npv=-4715

Доля дефолтников меньше в 2 раза - npv=5438

доля дефолтников влияет на loan_loss - чем выше доля тем выше потери банка

сумма recovery, получаемая от дефолтников, намного меньше чем потери банка


###`Доля полныx закрытий`

In [17]:
cur_dist = np.zeros(101)
dlq_dist = np.zeros(101)
clo_dist = np.zeros(101)
def_dist = np.zeros(101)

dlq_dist[1:12] += 0.03

def_dist[6:] += 0.02
def_dist[9:] += 0.03

def_dist[12:] += dlq_dist[11]
clo_dist[3:] += 0.30
clo_dist[6:] += 0.30
clo_dist[12:] = 1 - def_dist[12]

cur_dist = 1 - def_dist - clo_dist - dlq_dist

portfolio_distribution_c_big = {
    'CUR' : cur_dist,
    'DLQ' : dlq_dist,
    'DEF' : def_dist,
    'CLO' : clo_dist
         }

In [18]:
portfolio_distribution_c_big['CLO']

array([0.  , 0.  , 0.  , 0.3 , 0.3 , 0.3 , 0.6 , 0.6 , 0.6 , 0.6 , 0.6 ,
       0.6 , 0.92, 0.92, 0.92, 0.92, 0.92, 0.92, 0.92, 0.92, 0.92, 0.92,
       0.92, 0.92, 0.92, 0.92, 0.92, 0.92, 0.92, 0.92, 0.92, 0.92, 0.92,
       0.92, 0.92, 0.92, 0.92, 0.92, 0.92, 0.92, 0.92, 0.92, 0.92, 0.92,
       0.92, 0.92, 0.92, 0.92, 0.92, 0.92, 0.92, 0.92, 0.92, 0.92, 0.92,
       0.92, 0.92, 0.92, 0.92, 0.92, 0.92, 0.92, 0.92, 0.92, 0.92, 0.92,
       0.92, 0.92, 0.92, 0.92, 0.92, 0.92, 0.92, 0.92, 0.92, 0.92, 0.92,
       0.92, 0.92, 0.92, 0.92, 0.92, 0.92, 0.92, 0.92, 0.92, 0.92, 0.92,
       0.92, 0.92, 0.92, 0.92, 0.92, 0.92, 0.92, 0.92, 0.92, 0.92, 0.92,
       0.92, 0.92])

In [19]:
model.npv_calc(amount=500_000, term=12, rate=0.25, portfolio_distribution=portfolio_distribution_c_big)

np.int64(1186)

In [20]:
cur_dist = np.zeros(101)
dlq_dist = np.zeros(101)
clo_dist = np.zeros(101)
def_dist = np.zeros(101)

dlq_dist[1:12] += 0.05

def_dist[6:] += 0.03
def_dist[9:] += 0.03

def_dist[12:] += dlq_dist[11]
def_dist *= 1.3
clo_dist[3:] += 0.05
clo_dist[6:] += 0.05
clo_dist[12:] = 1 - def_dist[12]

cur_dist = 1 - def_dist - clo_dist - dlq_dist

portfolio_distribution_c_small = {
    'CUR' : cur_dist,
    'DLQ' : dlq_dist,
    'DEF' : def_dist,
    'CLO' : clo_dist
         }

In [21]:
portfolio_distribution_c_small['CLO']

array([0.   , 0.   , 0.   , 0.05 , 0.05 , 0.05 , 0.1  , 0.1  , 0.1  ,
       0.1  , 0.1  , 0.1  , 0.857, 0.857, 0.857, 0.857, 0.857, 0.857,
       0.857, 0.857, 0.857, 0.857, 0.857, 0.857, 0.857, 0.857, 0.857,
       0.857, 0.857, 0.857, 0.857, 0.857, 0.857, 0.857, 0.857, 0.857,
       0.857, 0.857, 0.857, 0.857, 0.857, 0.857, 0.857, 0.857, 0.857,
       0.857, 0.857, 0.857, 0.857, 0.857, 0.857, 0.857, 0.857, 0.857,
       0.857, 0.857, 0.857, 0.857, 0.857, 0.857, 0.857, 0.857, 0.857,
       0.857, 0.857, 0.857, 0.857, 0.857, 0.857, 0.857, 0.857, 0.857,
       0.857, 0.857, 0.857, 0.857, 0.857, 0.857, 0.857, 0.857, 0.857,
       0.857, 0.857, 0.857, 0.857, 0.857, 0.857, 0.857, 0.857, 0.857,
       0.857, 0.857, 0.857, 0.857, 0.857, 0.857, 0.857, 0.857, 0.857,
       0.857, 0.857])

In [22]:
model.npv_calc(amount=500_000, term=12, rate=0.25, portfolio_distribution=portfolio_distribution_c_small)

np.int64(-2169)

при увеличении доли полных закрытий, уменьшается доля дефолтников,  банк меньше теряет денег, отсюда и следует что NPV выше

###`Доля капитала в выдаче`

In [23]:
model = NPVModel()

In [24]:
for eq in [0.2, 0.125, 0.05]:
    model.eq_req = eq
    npv = model.npv_calc(amount=500_000, term=12, rate=0.25, portfolio_distribution=portfolio_distribution)
    print(f"eq_req={eq}: NPV={npv}")

eq_req=0.2: NPV=-1953
eq_req=0.125: NPV=361
eq_req=0.05: NPV=2676


eq_req = 0.20 -> NPV = -1953

eq_req = 0.125 -> NPV = 361

eq_req = 0.05 -> NPV = 2676


можно отследить, что при меньшей доли капитала NPV выше

По сути, чем меньше доля капитала тем выгоднее банку. Повышается значение ROE



```
        eq_req_curve = assets*self.eq_req
        fund_req_curve = assets*(1 - self.eq_req)
        # shareholders
        eq_req_ch = eq_req_curve - np.append(eq_req_curve[0], eq_req_curve[:-1])
        cf_to_shareholders = niat - np.append(eq_req_curve[0], eq_req_ch[1:])

```




в этом месте наблюдается зависимость на cf_to_shareholders

больше eq_req -> меньше cf_to_shareholders -> меньше значение pv

# Задание 2

**Добавить страховку**

*Условия страховки:*
- Страховка стоит 0.5% от лимита, выплаты ежемесячно
- Подключена у 40% клиентов в статусе CUR и у 60% в статусе DLQ
- Возможно наступление страхового случая с вероятностью 0.1% (в каждый месяц), при подключенной страховке мы должны возместить всю сумму текущей задолженности

В модель добавлены константы, добавлен расчет страховых выплат в profit. В функции loss добавлены потери при страховом случае

In [25]:
import numpy as np

class NPVModel:

    avg_missed_payments = 1.5
    recovery = 0.50
    dlq_penalty_amount = 500
    oper_costs = 100
    collection_costs = 600
    tax_rate = 0.20
    discounting_rate = 0.30
    eq_req = 0.125
    cost_of_funds = 0.16
    dlq_ratio = np.ones(101)*0.20
    dlq_ratio[0] = 0
    acquisition_cost = 1000
    # переменные для страховки
    insurance_rate = 0.005
    insurance_cur = 0.4
    insurance_dlq = 0.6
    insurance_accident_rate = 0.001

    def __init__(self):
        pass

    def model_balance_calculations(self, amount, rate, term):
        '''
        Расчет модельных баланса, выплаченных процентов, регулярного платежа
        :param amount: Сумма кредита
        :param rate: Ставка
        :param term: Срок
        :return:
        balance : np.array(101) : остаток тела долга по кредиту на каждый месяц
        interest : np.array(101) : выплата по процентам каждый месяц
        regular_payment : float : размер регулярного платежа
        '''

        regular_payment = round(amount * (rate / 12) *\
                                (1 + (rate / 12)) / (1 - ((1 + rate / 12) ** (-term))))
        # График баланса и процентов
        balance = np.zeros(101)
        interest = np.zeros(101)
        balance[0] = amount

        for i in range(1, term + 1):
            int_payment = balance[i - 1] * rate / 12
            debt_payment = regular_payment - int_payment
            balance[i] = max(0, round(balance[i - 1] - debt_payment))
            interest[i] = int_payment
        return balance, interest, regular_payment

    def distribution_calc(self, portfolio_distribution, pd=None, pa=None):
        '''
        Расчет распределения по статусам
        :param portfolio_distribution: график распределения по статусам
        :return:
        cur_dist : np.array(101) : доля клиентов в статусе CUR на каждый месяц
        dlq_dist : np.array(101) : доля клиентов в статусе DLQ на каждый месяц
        act_dist : np.array(101) : доля клиентов в статусе ACT на каждый месяц
        def_dist : np.array(101) : доля клиентов в статусе DEF на каждый месяц
        clo_dist : np.array(101) : доля клиентов в статусе CLO на каждый месяц
        '''

        cur_dist = portfolio_distribution['CUR']
        dlq_dist = portfolio_distribution['DLQ']
        act_dist = cur_dist + dlq_dist
        def_dist = portfolio_distribution['DEF']
        clo_dist = portfolio_distribution['CLO']


        return cur_dist, dlq_dist, act_dist, def_dist, clo_dist

    def cur_balance_calc(self, model_balance, cur_dist):
        '''
        Расчет модельного баланса в статусе CUR
        :param model_balance: плановый график баланса
        :param cur_dist: доля клиентов в статусе CUR на каждый месяц
        :return:
        principal_balance_cur : np.array(101) : principal balance в статусе CUR
        gross_balance_cur : np.array(101) : gross balance в статусе CUR
        '''
        principal_balance_cur = model_balance*cur_dist
        gross_balance_cur = model_balance*cur_dist
        return principal_balance_cur, gross_balance_cur

    def dlq_balance_calc(self, model_balance, regular_payment, dlq_dist):
        '''
        Расчет модельного баланса в статусе DLQ
        :param model_balance: плановый график баланса
        :param regular_payment: размер регулярного платежа
        :param dlq_dist: доля клиентов в статусе DLQ на каждый месяц
        :return:
        principal_balance_dlq : np.array(101) : principal balance в статусе DLQ
        gross_balance_dlq : np.array(101) : gross balance в статусе DLQ
        '''
        principal_balance_dlq = np.append(model_balance[1:], model_balance[-1])*dlq_dist
        gross_balance_dlq = (model_balance + regular_payment*self.avg_missed_payments)*dlq_dist
        return principal_balance_dlq, gross_balance_dlq

    def act_balance_calc(self, principal_balance_cur, principal_balance_dlq, gross_balance_cur, gross_balance_dlq):
        '''
        Расчет модельного баланса в статусе ACT
        :param principal_balance_cur: principal balance в статусе CUR
        :param principal_balance_dlq: principal balance в статусе DLQ
        :param gross_balance_cur: principal balance в статусе CUR
        :param gross_balance_dlq: principal balance в статусе DLQ
        :param dlq_dist: доля клиентов в статусе DLQ на каждый месяц
        :return:
        principal_balance_act : np.array(101) : principal balance в статусе ACT
        gross_balance_act : np.array(101) : gross balance в статусе ACT
        '''
        principal_balance_act = principal_balance_cur + principal_balance_dlq
        gross_balance_act = gross_balance_cur + gross_balance_dlq
        return principal_balance_act, gross_balance_act

    def def_balance_calc(self, model_balance, regular_payment, def_dist):
        '''
        Расчет модельного баланса в статусе DEF
        :param model_balance: плановый график баланса
        :param regular_payment: размер регулярного платежа
        :param def_dist: доля клиентов в статусе DEF на каждый месяц
        :return:
        principal_balance_def : np.array(101) : principal balance в статусе DEF
        gross_balance_def : np.array(101) : gross balance в статусе DEF
        '''
        principal_balance_def = np.zeros(101)
        gross_balance_def = np.zeros(101)

        def_dist_change = def_dist[4:] - def_dist[3:-1]
        principal_balance_def[4:] = np.cumsum(model_balance[:-4] * def_dist_change)
        gross_balance_def[4:] = np.cumsum((model_balance[4:] + 4 * regular_payment) * def_dist_change)

        return principal_balance_def, gross_balance_def

    def profit_calc(self, principal_balance_act, principal_balance_def, term, rate, dlq_dist, cur_dist, amount):
        '''

        :param principal_balance_act: principal balance в статусе ACT
        :param principal_balance_def: principal balance в статусе DEF
        :param term: срок кредита
        :param rate: ставка по кредиту
        :param dlq_dist: доля клиентов в статусе DLQ на каждый месяц
        :return:
        profit : np.array(101) : доход на каждый месяц
        '''
        interest_profit = principal_balance_act*rate/12
        penatly_profit = dlq_dist*self.dlq_penalty_amount

        new_def_balance = np.append(principal_balance_def[1:], 0) - principal_balance_def
        recovery_profit = new_def_balance*self.recovery
        # клиенты со страховкой выплачивают дополнительно процент от лимита ежемесячно
        insurance_cost_month = self.insurance_rate * amount
        insurance_profit = (self.insurance_cur * cur_dist + self.insurance_dlq * dlq_dist) * insurance_cost_month

        profit = interest_profit + recovery_profit + penatly_profit + insurance_profit
        profit[term+1:] = 0

        return profit

    def loss_calc(self, gross_balance_act, principal_balance_def, principal_balance_cur, principal_balance_dlq, term, act_dist, dlq_dist, def_dist):
        '''

        :param gross_balance_act: gross balance в статусе ACT
        :param principal_balance_def: principal balance в статусе DEF
        :param term: срок кредита
        :param act_dist: доля клиентов в статусе ACT на каждый месяц
        :param dlq_dist: доля клиентов в статусе DLQ на каждый месяц
        :param def_dist: доля клиентов в статусе DEF на каждый месяц
        :return:
        loss : np.array(101) : лосс на каждый месяц
        '''
        oper_loss = np.append(act_dist[1:], 0)*self.oper_costs
        loan_loss =  principal_balance_def - np.append(0, principal_balance_def[:-1])
        collection_loss = (np.append(dlq_dist[1:], 0) + np.append(def_dist[1:], 0))*self.collection_costs

        prev_gross_balance_act = np.append(gross_balance_act[0], gross_balance_act[1:])
        cost_of_funds_loss = prev_gross_balance_act*(1 - self.eq_req)*self.cost_of_funds/12
        # потери при страховом случае: возмещается сумма долга
        insurance_accident = self.insurance_accident_rate * (self.insurance_cur * principal_balance_cur + self.insurance_dlq * principal_balance_dlq)
        loss = loan_loss + cost_of_funds_loss + oper_loss + collection_loss + insurance_accident
        loss[term+1:] = 0

        return loss, loan_loss, cost_of_funds_loss, oper_loss, collection_loss

    def assets_liabilities_calc(self, gross_balance_act):
        '''

        :param gross_balance_act: gross balance в статусе ACT
        :return:
        assets : np.array(101) : активы в проекте
        eq_req_curve : np.array(101) : активы, обеспеченные капиталом
        fund_req_curve : np.array(101) : активы, обеспеченные фондами
        '''

        # assets
        assets = gross_balance_act

        #liabilities
        eq_req_curve = assets*self.eq_req
        fund_req_curve = assets*(1 - self.eq_req)

        return assets, eq_req_curve, fund_req_curve

    def niat_calc(self, profit, loss):
        '''
        Расчет NIAT
        :param profit: суммарный доход на каждый месяц
        :param loss: суммарный лосс на каждый месяц
        :return:
        nibt : np.array(101) : прибыль до налогообложения
        niat : np.array(101) : прибыль после налогообложения
        tax : np.array(101) : налог в каждом месяце
        '''

        nibt = profit - loss
        tax = nibt*self.tax_rate
        niat = nibt - tax

        return nibt, niat, tax

    def cashflow_calc(self, principal_balance_act, principal_balance_def, amount, profit,
                      cost_of_funds_loss, oper_costs, collection_costs, tax, niat, eq_req_curve, fund_req_curve):
        '''
        Расчет денежных потоков
        :param principal_balance_act: principal balance в статусе DEF
        :param principal_balance_def: principal balance в статусе DEF
        :param amount: срок кредита
        :param profit: доход на каждый месяц
        :param cost_of_funds_loss: лоссы на фондирование
        :param oper_costs: операционные расходы
        :param collection_costs: расходы на коллекшн
        :param tax: налог в каждом месяце
        :param niat: доход после налогообложения
        :param eq_req_curve: активы, обеспеченные капиталом
        :param fund_req_curve: активы, обеспеченные фондами
        :return:
        cf_to_client : np.array(101) : денежный поток к клиенту
        cf_to_shareholders : np.array(101) : денежный поток к акционерам
        cf_to_debtholders : np.array(101) : денежный поток к фондам
        cf_to_cost_and_tax : np.array(101) : денежный поток на косты и налоги
        '''
        # client
        delta_principal_balance_act = (np.append(0, principal_balance_act[:100]) - principal_balance_act)
        delta_principal_balance_def = (np.append(0, principal_balance_def[:100]) - principal_balance_def)
        repayments = delta_principal_balance_act - delta_principal_balance_def + profit
        cf_to_client = np.append(amount, -repayments[1:])

        # debtoholder
        fund_req_ch = fund_req_curve - np.append(fund_req_curve[0], fund_req_curve[:-1])
        fund_req_change = np.append(fund_req_curve[0], fund_req_ch[1:])
        cf_to_debtholders = cost_of_funds_loss - fund_req_change

        # shareholders
        eq_req_ch = eq_req_curve - np.append(eq_req_curve[0], eq_req_curve[:-1])
        cf_to_shareholders = niat - np.append(eq_req_curve[0], eq_req_ch[1:])

        # cost and tax
        cf_to_cost_and_tax = oper_costs + collection_costs + tax

        return cf_to_client, cf_to_shareholders, cf_to_debtholders, cf_to_cost_and_tax

    def npv_calc(self, amount, rate, term, portfolio_distribution, pd=None, pa=None):
        '''
        Расчет NPV
        :param amount: сумма кредита
        :param rate: ставка по кредиту
        :param term: срок кредита
        :param portfolio_distribution: график распределения по статусам
        :param pd: вероятность дефолта к 12-ому месяцу
        :param pa: вероятность полного досрочного погашения к 6-ому месяцу
        :return: npv: чистая приведенная стоимость кредита
        '''

        # Считаем балансы
        model_balance, interest, regular_payment =  self.model_balance_calculations(amount, rate, term)

        # Считаем распределение
        cur_dist, dlq_dist, act_dist, def_dist, clo_dist = self.distribution_calc(portfolio_distribution)

        # Считаем балансы
        principal_balance_cur, gross_balance_cur = self.cur_balance_calc(model_balance, cur_dist)
        principal_balance_dlq, gross_balance_dlq = self.dlq_balance_calc(model_balance, regular_payment, dlq_dist)
        principal_balance_act, gross_balance_act = self.act_balance_calc(principal_balance_cur, principal_balance_dlq, gross_balance_cur, gross_balance_dlq)
        principal_balance_def, gross_balance_def = self.def_balance_calc(model_balance, regular_payment, def_dist)

        # Считаем профиты
        profit = self.profit_calc(principal_balance_act, principal_balance_def, term, rate, dlq_dist, cur_dist, amount)
        loss, loan_loss, cost_of_funds_loss, oper_loss, collection_loss = self.loss_calc(gross_balance_act, principal_balance_def,principal_balance_cur, principal_balance_dlq,
                                                                                                                  term, act_dist, dlq_dist, def_dist)
        nibt, niat, tax = self.niat_calc(profit, loss)

        # Считаем активы и денежные потоки
        assets, eq_req_curve, fund_req_curve = self.assets_liabilities_calc(gross_balance_act)
        cf_to_client, cf_to_shareholders, cf_to_debtholders, cf_to_cost_and_tax =\
            self.cashflow_calc(principal_balance_act, principal_balance_def, amount, profit, cost_of_funds_loss, oper_loss, collection_loss,
                               tax, niat, eq_req_curve, fund_req_curve)


        # Считаем NPV
        disc_curve = np.array([1 / ((1 + self.discounting_rate) ** (i / 12.)) for i in range(101)])
        pv = np.round(np.sum(disc_curve * cf_to_shareholders)).astype(int)
        npv = pv - self.acquisition_cost

        return npv

In [26]:
cur_dist = np.zeros(101)
dlq_dist = np.zeros(101)
clo_dist = np.zeros(101)
def_dist = np.zeros(101)

dlq_dist[1:12] += 0.05

def_dist[6:] += 0.03
def_dist[9:] += 0.03

def_dist[12:] += dlq_dist[11]

clo_dist[3:] += 0.10
clo_dist[6:] += 0.10
clo_dist[12:] = 1 - def_dist[12]

cur_dist = 1 - def_dist - clo_dist - dlq_dist

portfolio_distribution = {
    'CUR' : cur_dist,
    'DLQ' : dlq_dist,
    'DEF' : def_dist,
    'CLO' : clo_dist
         }

In [27]:
model = NPVModel()
model.npv_calc(amount=500_000, term=12, rate=0.25, portfolio_distribution=portfolio_distribution)

np.int64(6996)

При добавлении страховки значение npv выросло в разы: 6996 > 361
Такой рост можно объяснить высокой выплатой за страховку ( 0.5% от лимита ежемесячно)
А также, из-за низкой вероятности наступления страхового случая, потери банка малы относительно прибыли